In [1]:
using DataFrames, CSV, Query, StatsBase

In [2]:
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20;

Loading ISCN Layers dataset

In [3]:
df_ISCN_orig = CSV.read("ISCN_SOC-DATA_LAYER_1-1.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 93 / 94 columns around data row: 227765. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,dataset_name_sub,dataset_name_soc,lat (dec. deg),long (dec. deg),datum (datum),state (state_province),country (country),site_name,observation_date (YYYY-MM-DD),profile_name,layer_name,layer_top (cm),layer_bot (cm),hzn_desgn_other,hzn,hzn_desgn,layer_note,color,vegclass_local,soil_taxon,soil_series,bd_method,bd_samp (g cm-3),bd_tot (g cm-3),bd_whole (g cm-3),bd_other (g cm-3),bdNRCS_prep_code,cNRCS_prep_code,c_method,c_tot (percent),oc (percent),loi (percent),n_tot (percent),c_to_n (mass ratio),soc (g cm-2),soc_carbon_flag,soc_method,ph_method,ph_cacl,ph_h2o,ph_other,caco3 (percent),sand_tot_psa (percent),silt_tot_psa (percent),clay_tot_psa (percent),wpg2_method,wpg2 (percent),cat_exch (cmol H+ kg-1),al_dith (specified by al_fe_units),al_ox (specified by al_fe_units),al_other (specified by al_fe_units),fe_dith (specified by al_fe_units)
,String,String,String?,String?,String?,String?,String?,String,String?,String,String,Int64?,Int64?,Missing,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,Missing,String?,String?,String?,String?,String?,String?,String?,String?,Int64?,Int64?,Int64?,String?,Int64?,String?,String?,String?,String?,String?
1,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05963,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,27",Eqn 11 + NRCS (c---w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05964,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,27",Eqn 7 + NRCS (c---w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05965,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,12",noFill + NRCS (c-s-w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
4,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05969,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,02",noFill + NRCS (c-s-w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05970,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,02",noFill + NRCS (c-s-w),missing,missing,mis

In [4]:
count(ismissing,(df_ISCN_orig[:,:hzn_desgn_other])) #227764 - all missing values, not using this column
count(ismissing,(df_ISCN_orig[:,:hzn_desgn]))  		#13812  - ok
count(ismissing,(df_ISCN_orig[:,:hzn])) 			#12244  - ok

12244

In [5]:
function tidy_names(old_names)

return new_names = old_names |>
    #remove leading and trailing spaces
    n -> strip.(n) |>

    #replace spaces with underscores
    n -> replace.(n, ' ' => '_') |> 

    #remove parenthesis
    n -> replace.(n,'(' => "") |> 
    n -> replace.(n,')' => "") |>

    #remove dashes and dots
    n -> replace.(n,'-' => "") |> 
    n -> replace.(n, '.' => "") |>

    #all lowercase
    n -> lowercase.(n)

end

tidy_names (generic function with 1 method)

In [6]:
#copy of original dataset and tidied up column names
df_ISCN = deepcopy(df_ISCN_orig)
rename!(df_ISCN, names(df_ISCN) .=> tidy_names(names(df_ISCN)))

#define new column that has unique key, site_name_profile_name
df_ISCN.sp_name = [(row.site_name * "_" * row.profile_name) for row in eachrow(df_ISCN)]

df_ISCN = df_ISCN |>

#pick relevant columns
@select(:sp_name,:layer_name,:lat_dec_deg,:long_dec_deg,
        :observation_date_yyyymmdd,:hzn,:hzn_desgn,:layer_top_cm,:layer_bot_cm,:oc_percent) |>

#remove layers with missing info
@dropna(:layer_top_cm,:layer_bot_cm,:oc_percent) |>

#keep layers for which top < bot
@filter(_.layer_top_cm < _.layer_bot_cm) |>

#parse oc_percent values and keep only non-negative ones
@mutate(oc_percent = replace(_.oc_percent,','=>'.')) |>
@mutate(oc_percent = parse(Float64, _.oc_percent)) |>
@filter(_.oc_percent >= 0) |>

#parse latitude values
#missings are replaced with -300 to allow parsing, but we keep layers with missing lat/long
@replacena(:lat_dec_deg=>"-300") |>
@mutate(lat_dec_deg = replace(_.lat_dec_deg,','=>'.')) |>
@mutate(lat_dec_deg = parse(Float64, _.lat_dec_deg)) |>

#parse longitude values 
@replacena(:long_dec_deg=>"-300") |>
@mutate(long_dec_deg = replace(_.long_dec_deg,','=>'.')) |>
@mutate(long_dec_deg = parse(Float64, _.long_dec_deg)) |>

@rename(:observation_date_yyyymmdd => :obs_date_yyyymmdd) |> 

#keep only unique layers
@unique() |>

DataFrame

end

LoadError: syntax: unexpected "end"

In [7]:
function continuity_check(profile) 
    #the function assumes layers are ordered (e.g.  0 -> 20 -> 40)

    top = profile.layer_top_cm
    bot = profile.layer_bot_cm

    num_layers = length(top)

    #1st layer is always good
    mask = [true]

    #from 2nd layer downward, we check for matching depths at bot/top
    mask = append!(mask, [top[l] == bot[l-1] for l in 2:num_layers])

    #if we run into a non-matching layer, we mark as 'false' all following ones
    first_noMatch_id = findfirst(!, mask)

    if !isnothing(first_noMatch_id) #true if a non-contiguous layer has been found
        mask[first_noMatch_id:end] .= false
    end

    return mask	
end

continuity_check (generic function with 1 method)

In [8]:
function continuity_check_df(df)
#generate continuity mask for the whole dataframe
layers_keep = Bool[]

for profile in unique(df.sp_name)

    append!(layers_keep,continuity_check(df[df.sp_name.==profile,:]))

end

return layers_keep

end

continuity_check_df (generic function with 1 method)

In [9]:
df_ISCN_cont = df_ISCN[continuity_check_df(df_ISCN),:] #df with only continuous profiles

,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,hzn,hzn_desgn,layer_top_cm,layer_bot_cm,oc_percent
,String,String,Float64,Float64,String?,String?,String?,Int64,Int64,Float64
1,50ND075005_40A0001,40A00001,48.805,-101.735,19/09/1950,A,Ap,0,23,2.99
2,50ND075005_40A0001,40A00002,48.805,-101.735,19/09/1950,A,A,23,33,0.93
3,50ND075005_40A0001,40A00003,48.805,-101.735,19/09/1950,B,Bw,33,48,0.7
4,50ND075005_40A0001,40A00004,48.805,-101.735,19/09/1950,B,Bk,48,97,0.34
5,50ND075005_40A0001,40A00005,48.805,-101.735,19/09/1950,B,B21,97,122,0.2
6,50ND075005_40A0001,40A00006,48.805,-101.735,19/09/1950,B,B22,122,145,0.18
7,50ND075005_40A0001,40A00007,48.805,-101.735,19/09/1950,B,By,145,183,0.15
8,50ND075003_40A0013,40A00094,48.9366,-101.585,18/09/1950,A,Ap,0,15,2.42
9,50ND075003_40A0013,40A00095,48.9366,-101.585,18/09/1950,B,Bw1,15,25,0.98


Importing BD values is necessary before inferring layer designation

In [10]:
df_BD_pred_orig = CSV.read("ISCNTemplate_NRCS_BD_predictions.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 26 / 27 columns around data row: 434583. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,dataset_name,site_name,profile_name,layer_name,layer_top,layer_bot,hzn_desgn,c_method,bd_method,bd_pred_1,bd_pred_2,bd_samp,bd_tot,bd_whole,bd_other,bdNRCS_prep_code,sand_tot_psa,silt_tot_psa,clay_tot_psa,wpg2_method,wpg2,soc,soc_sigma,soc_method,c_tot,oc,cNRCS_prep_code
,String,String,String,String,String,String,String?,String?,String?,String,String,String,String,String,String,String?,String,String,String,String?,String,String,String,String,String,String,String?
1,Boby_Mack,BF-61,BF-61-1,BF61-1-F,0,4,unknown,missing,missing,"1,14","0,73","0,04",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"44,62",NA,missing
2,Boby_Mack,BF-61,BF-61-1,BF61-1-H,4,6,unknown,missing,missing,"1,14","0,74","0,12",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"30,21",NA,missing
3,Boby_Mack,BF-61,BF-61-1,BF61-1-M1,6,11,unknown,missing,missing,"1,27","1,03","1,15",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"5,32",NA,missing
4,Boby_Mack,BF-61,BF-61-1,BF61-1-M2,11,16,unknown,missing,missing,"1,31","1,13","1,12",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"3,88",NA,missing
5,Boby_Mack,BF-61,BF-61-2,BF61-2-DM,0,5,o,missing,missing,"0,56","0,36","0,01",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"45,53",NA,missing
6,Boby_Mack,BF-61,BF-61-2,BF61-2-F,5,19,unknown,missing,missing,"1,34","0,67","0,09",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"48,79",NA,missing
7,Boby_Mack,BF-61,BF-61-2,BF61-2-H,19,22,unknown,missing,missing,"1,29","0,77","0,48",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"19,17",NA,missing
8,Boby_Mack,BF-61,BF-61-2,BF61-2-H:354,22,25,unknown,missing,missing,"1,28","0,67","0,12",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"44,01",NA,missing
9,Boby_Mack,BF-61,BF-61-2,BF61-2-M1,25,30,unknown,missing,missing,"1,31","0,96","1,04",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"5,65",NA,missing


In [11]:
#copy of original dataset and tidied up column names
df_BD_pred = deepcopy(df_BD_pred_orig)

#define new column that has unique key, site_name_profile_name
df_BD_pred.sp_name = [(row.site_name * "_" * row.profile_name) for row in eachrow(df_BD_pred)]

df_BD_pred = df_BD_pred |>

#pick relevant columns
@select(:sp_name,:layer_name,:bd_pred_1,:bd_pred_2) |>

##remove layers with missing predicted BD
@dropna(:bd_pred_1, :bd_pred_2) |>

#parse bd_pred_1 values and keep only non-negative ones
@mutate(bd_pred_1 = replace(_.bd_pred_1,','=>'.')) |>
@mutate(bd_pred_1 = parse(Float64, _.bd_pred_1)) |>
@filter(_.bd_pred_1 > 0) |>

#same with bd_pred_2 values
@mutate(bd_pred_2 = replace(_.bd_pred_2,','=>'.')) |>
@mutate(bd_pred_2 = parse(Float64, _.bd_pred_2)) |>
@filter(_.bd_pred_2 > 0) |>

#keep only unique sites/profiles/layers
@unique((_.sp_name, _.layer_name)) |>

#‘bd_pred_1’ to be used for organic horizons, ‘be_pred_2’ for mineral soil horizons
@rename(:bd_pred_1 => :bd_organic) |>
@rename(:bd_pred_2 => :bd_mineral) |>

DataFrame

,sp_name,layer_name,bd_organic,bd_mineral
,String,String,Float64,Float64
1,BF-61_BF-61-1,BF61-1-F,1.14,0.73
2,BF-61_BF-61-1,BF61-1-H,1.14,0.74
3,BF-61_BF-61-1,BF61-1-M1,1.27,1.03
4,BF-61_BF-61-1,BF61-1-M2,1.31,1.13
5,BF-61_BF-61-2,BF61-2-DM,0.56,0.36
6,BF-61_BF-61-2,BF61-2-F,1.34,0.67
7,BF-61_BF-61-2,BF61-2-H,1.29,0.77
8,BF-61_BF-61-2,BF61-2-H:354,1.28,0.67
9,BF-61_BF-61-2,BF61-2-M1,1.31,0.96


In [12]:
function infer_designation(old_desig, blk)
#takes layer and returns its inferred soil designation, either 'organic' or 'mineral'

#old_desig = get(existing_desig)
new_desig = "no_desig" #value to be eventually returned

#blk = get(bulk_density) #already parsed to F64, no missing values assumed
threshold = 0.5

control_desig = ['O','o','H','h'] #possible chars in existing desig that indicate presence of organic material in the layer

new_desig_organic = "organic"
new_desig_mineral = "mineral"

if old_desig == ""
    #check on bulk density value to decide which designation applies 
    new_desig = (blk <= threshold) ? new_desig_organic : new_desig_mineral

#designation not missing			
else	
    check_desig = [in(c,control_desig) for c in old_desig] #check if any char in the existing designation matches organic control
    check_desig_BD = !isnothing(findfirst(check_desig)) & (blk <= threshold) #true if any char matches and BD < treshold

    new_desig = check_desig_BD ? new_desig_organic : new_desig_mineral
end


return new_desig
end

infer_designation (generic function with 1 method)

In [13]:
#adding predicted BD values to each layer
df_hz = innerjoin(df_ISCN_cont, df_BD_pred, on = [:sp_name, :layer_name])	
unique!(df_hz, [:sp_name, :layer_name])
	
#work on layer designation
df_hz.hzn = replace(df_hz.hzn, "?" => "") 
df_hz.hzn = replace(df_hz.hzn, missing => "")
	
df_hz.hzn_desgn = replace(df_hz.hzn_desgn, "unknown" => "") 
df_hz.hzn_desgn = replace(df_hz.hzn_desgn, missing => "") 

#create new column with concatenated designations, but leave row empty if both are empty
df_hz.hzn_merged = [!isempty(row.hzn * row.hzn_desgn) ? (row.hzn * "_" * row.hzn_desgn) : "" for row in eachrow(df_hz)]
	
#infer layer designation and write to new col, using only bd_organic for threshold check
df_hz.infer_DESIG = [infer_designation(r.hzn_merged,r.bd_organic) for r in eachrow(df_hz) ]	

#getting rid of old designation cols
select!(df_hz, Not(:hzn))	
select!(df_hz, Not(:hzn_desgn))	


,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,layer_top_cm,layer_bot_cm,oc_percent,bd_organic,bd_mineral,hzn_merged,infer_DESIG
,String,String,Float64,Float64,String?,Int64,Int64,Float64,Float64,Float64,String,String
1,50ND075005_40A0001,40A00001,48.805,-101.735,19/09/1950,0,23,2.99,1.36,1.44,A_Ap,mineral
2,50ND075005_40A0001,40A00002,48.805,-101.735,19/09/1950,23,33,0.93,1.24,1.36,A_A,mineral
3,50ND075005_40A0001,40A00003,48.805,-101.735,19/09/1950,33,48,0.7,1.35,1.38,B_Bw,mineral
4,50ND075005_40A0001,40A00004,48.805,-101.735,19/09/1950,48,97,0.34,1.43,1.36,B_Bk,mineral
5,50ND075005_40A0001,40A00005,48.805,-101.735,19/09/1950,97,122,0.2,1.43,1.31,B_B21,mineral
6,50ND075005_40A0001,40A00006,48.805,-101.735,19/09/1950,122,145,0.18,1.35,1.32,B_B22,mineral
7,50ND075005_40A0001,40A00007,48.805,-101.735,19/09/1950,145,183,0.15,1.41,1.34,B_By,mineral
8,50ND075003_40A0013,40A00094,48.9366,-101.585,18/09/1950,0,15,2.42,1.46,1.5,A_Ap,mineral
9,50ND075003_40A0013,40A00095,48.9366,-101.585,18/09/1950,15,25,0.98,1.34,1.36,B_Bw1,mineral


In [14]:
function depth_clip(profile; depth_min=0, depth_max=100)
#takes in a profile with N layers and returns a clipped profile, by default from 0 to 100cm
#run this function ALWAYS AFTER checking for layers continuity


top = profile.layer_top_cm
bot = deepcopy(profile.layer_bot_cm) #create copy of the original profile bot, to be edited later

n_layers = length(top)

#init array of 'true' -> keep all layers unless proven guilty
layers_keep = Vector{Bool}(undef,n_layers)
layers_keep .= true

#if profile doesn't reach threshold, or starts after min, ditch the whole profile
if maximum(bot) < depth_max || minimum(top) > depth_min; 	

    layers_keep .= false

#othewise mark layers to drop above threshold, and clip the overlapping one
else 
    for lr in 1:n_layers
        #simple case, both top and bot below threshold
        if top[lr] >= depth_max; layers_keep[lr] = false; end

        #layers overlapping threshold
        if top[lr] < depth_max && bot[lr] > depth_max
            bot[lr] = depth_max
        end

    end

    profile.layer_bot_cm = bot

end


return profile[layers_keep,:]

end

depth_clip (generic function with 1 method)

In [15]:
function compute_clipped_profiles(df)
#initialize empty DF with same columns and types as starting one
results = DataFrame(eltypes(df), names(df))

for profile in unique(df.sp_name)

	append!(results, depth_clip(df[df.sp_name .== profile,:]))

end

return results

end

compute_clipped_profiles (generic function with 1 method)

In [16]:
df_hz_clipped = compute_clipped_profiles(df_hz)

,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,layer_top_cm,layer_bot_cm,oc_percent,bd_organic,bd_mineral,hzn_merged,infer_DESIG
,String,String,Float64,Float64,String?,Int64,Int64,Float64,Float64,Float64,String,String
1,50ND075005_40A0001,40A00001,48.805,-101.735,19/09/1950,0,23,2.99,1.36,1.44,A_Ap,mineral
2,50ND075005_40A0001,40A00002,48.805,-101.735,19/09/1950,23,33,0.93,1.24,1.36,A_A,mineral
3,50ND075005_40A0001,40A00003,48.805,-101.735,19/09/1950,33,48,0.7,1.35,1.38,B_Bw,mineral
4,50ND075005_40A0001,40A00004,48.805,-101.735,19/09/1950,48,97,0.34,1.43,1.36,B_Bk,mineral
5,50ND075005_40A0001,40A00005,48.805,-101.735,19/09/1950,97,100,0.2,1.43,1.31,B_B21,mineral
6,50ND075003_40A0013,40A00094,48.9366,-101.585,18/09/1950,0,15,2.42,1.46,1.5,A_Ap,mineral
7,50ND075003_40A0013,40A00095,48.9366,-101.585,18/09/1950,15,25,0.98,1.34,1.36,B_Bw1,mineral
8,50ND075003_40A0013,40A00096,48.9366,-101.585,18/09/1950,25,46,0.76,1.41,1.37,B_Bw2,mineral
9,50ND075003_40A0013,40A00097,48.9366,-101.585,18/09/1950,46,100,0.27,1.41,1.36,B_Bk,mineral


#### For the ISCN dataset, organic carbon stock can be calculated as:

( 100cm * 100cm * layer\_height [cm] * bulk\_density [g/cm³] * oc_percent / 100 ) / 1000 

ORG carbon stock in [Kg]

In [17]:
function orgc_storage_desig(profile) 

id = profile.sp_name[1]
top = profile.layer_top_cm
bot = profile.layer_bot_cm
des = profile.infer_DESIG

org_bd = profile.bd_organic #[g/cm³]
min_bd = profile.bd_mineral

orgc = profile.oc_percent #[%]

layer_height = (bot.-top)
num_layers = length(top)

orgc_stored_mineral = 0.0
orgc_stored_organic = 0.0


for l in 1:num_layers
    #using different BD values depending on layer designation
    if des[l] == "organic"; orgc_stored_organic += ( 100 * 100 * layer_height[l] ) * ( org_bd[l] * orgc[l] / 100 ) / 1000; end
    if des[l] == "mineral"; orgc_stored_mineral += ( 100 * 100 * layer_height[l] ) * ( min_bd[l] * orgc[l] / 100 ) / 1000; end

end

return DataFrame(profile_id = id, 
                 orgc_stored_mineral = orgc_stored_mineral, 
                 orgc_stored_organic = orgc_stored_organic)
end

orgc_storage_desig (generic function with 1 method)

In [18]:
function compute_orgc_storage_desig(df)

    unique_profiles = unique(df.sp_name)

    #initialize result var with 1st call to function
    result = orgc_storage_desig(df[df.sp_name .== unique_profiles[1],:])

    for profile in unique_profiles[2:end]

        append!(result, orgc_storage_desig(df[df.sp_name .== profile,:]))

    end

    return result
end

compute_orgc_storage_desig (generic function with 1 method)

In [19]:
#compute stored ORGC for continuous, not clipped, profiles
df_storage_cont = compute_orgc_storage_desig(df_hz)

,profile_id,orgc_stored_mineral,orgc_stored_organic
,String,Float64,Float64
1,50ND075005_40A0001,16.8477,0.0
2,50ND075003_40A0013,13.095,0.0
3,50ND075007_40A0067,11.2555,0.0
4,59ND045001_40A0102,9.00305,0.0
5,50ND075006_40A0107,20.3409,0.0
6,50ND075008_40A0111,20.3598,0.0
7,50ND075009_40A0130,15.3723,0.0
8,50ND075010_40A0131,17.5185,0.0
9,S1949CO049005_40A0138,4.76136,0.0


In [20]:
#compute stored ORGC for continuous and clipped profiles
df_storage_clipped = compute_orgc_storage_desig(df_hz_clipped);

In [21]:
#profiles' info from the original dataset
df_info = df_hz[:,[:sp_name, :lat_dec_deg, :long_dec_deg, :obs_date_yyyymmdd]]
rename!(df_info, :sp_name => :profile_id )
unique!(df_info)

,profile_id,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd
,String,Float64,Float64,String?
1,50ND075005_40A0001,48.805,-101.735,19/09/1950
2,50ND075003_40A0013,48.9366,-101.585,18/09/1950
3,50ND075007_40A0067,48.8083,-101.707,19/09/1950
4,59ND045001_40A0102,46.4494,-98.37,21/08/1959
5,50ND075006_40A0107,48.8211,-101.747,19/09/1950
6,50ND075008_40A0111,48.8991,-101.554,20/09/1950
7,50ND075009_40A0130,48.9427,-101.587,20/09/1950
8,50ND075010_40A0131,48.9153,-101.606,20/09/1950
9,S1949CO049005_40A0138,-300.0,-300.0,02/09/1949


In [22]:
function depth_desig(profile) 

    id = profile.sp_name[1]
    des = profile.infer_DESIG
    top = profile.layer_top_cm
    bot = profile.layer_bot_cm

    missing_val = -1

    num_layers = length(top)

    mineral_index = [i for i in 1:num_layers if des[i] == "mineral"]
    organic_index = [i for i in 1:num_layers if des[i] == "organic"]

    #values to return

    profile_top = minimum(top)
    profile_bot = maximum(bot)

    mineral_top = isempty(mineral_index) ? missing_val : minimum(top[mineral_index])
    mineral_bot = isempty(mineral_index) ? missing_val : maximum(bot[mineral_index])

    organic_top = isempty(organic_index) ? missing_val : minimum(top[organic_index])
    organic_bot = isempty(organic_index) ? missing_val : maximum(bot[organic_index])


    return DataFrame(profile_id = id, 
                     profile_top = profile_top,
                     profile_bot = profile_bot,
                     organic_top = organic_top,
                     organic_bot = organic_bot,
                     mineral_top = mineral_top,
                     mineral_bot = mineral_bot)


end

depth_desig (generic function with 1 method)

In [23]:
function compute_depth_desig(df)

    unique_profiles = unique(df.sp_name)

    #initialize results with 1st call to function
    result = depth_desig(df[df.sp_name .== unique_profiles[1],:])

    for profile in unique_profiles[2:end]

        append!(result, depth_desig(df[df.sp_name .== profile,:]))

    end

    return result
end

compute_depth_desig (generic function with 1 method)

In [24]:
#designations depths per profile (non clipped)
df_cont_depth_desig = compute_depth_desig(df_hz)

,profile_id,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,50ND075005_40A0001,0,183,-1,-1,0,183
2,50ND075003_40A0013,0,213,-1,-1,0,213
3,50ND075007_40A0067,0,183,-1,-1,0,183
4,59ND045001_40A0102,0,124,-1,-1,0,124
5,50ND075006_40A0107,0,229,-1,-1,0,229
6,50ND075008_40A0111,0,203,-1,-1,0,203
7,50ND075009_40A0130,0,183,-1,-1,0,183
8,50ND075010_40A0131,0,183,-1,-1,0,183
9,S1949CO049005_40A0138,0,71,-1,-1,0,71


In [25]:
#designations depths per clipped profiles 
df_clipped_depth_desig = compute_depth_desig(df_hz_clipped);

In [26]:
#joining results with extra info	
df_cont_results = innerjoin(df_storage_cont, df_cont_depth_desig, df_info, on = :profile_id)

,profile_id,orgc_stored_mineral,orgc_stored_organic,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd
,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String?
1,50ND075005_40A0001,16.8477,0.0,0,183,-1,-1,0,183,48.805,-101.735,19/09/1950
2,50ND075003_40A0013,13.095,0.0,0,213,-1,-1,0,213,48.9366,-101.585,18/09/1950
3,50ND075007_40A0067,11.2555,0.0,0,183,-1,-1,0,183,48.8083,-101.707,19/09/1950
4,59ND045001_40A0102,9.00305,0.0,0,124,-1,-1,0,124,46.4494,-98.37,21/08/1959
5,50ND075006_40A0107,20.3409,0.0,0,229,-1,-1,0,229,48.8211,-101.747,19/09/1950
6,50ND075008_40A0111,20.3598,0.0,0,203,-1,-1,0,203,48.8991,-101.554,20/09/1950
7,50ND075009_40A0130,15.3723,0.0,0,183,-1,-1,0,183,48.9427,-101.587,20/09/1950
8,50ND075010_40A0131,17.5185,0.0,0,183,-1,-1,0,183,48.9153,-101.606,20/09/1950
9,S1949CO049005_40A0138,4.76136,0.0,0,71,-1,-1,0,71,-300.0,-300.0,02/09/1949


In [27]:
df_clipped_results = innerjoin(df_storage_clipped, df_clipped_depth_desig, df_info, on = :profile_id)

,profile_id,orgc_stored_mineral,orgc_stored_organic,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd
,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String?
1,50ND075005_40A0001,14.961,0.0,0,100,-1,-1,0,100,48.805,-101.735,19/09/1950
2,50ND075003_40A0013,10.9472,0.0,0,100,-1,-1,0,100,48.9366,-101.585,18/09/1950
3,50ND075007_40A0067,9.16548,0.0,0,100,-1,-1,0,100,48.8083,-101.707,19/09/1950
4,59ND045001_40A0102,8.89345,0.0,0,100,-1,-1,0,100,46.4494,-98.37,21/08/1959
5,50ND075006_40A0107,17.5183,0.0,0,100,-1,-1,0,100,48.8211,-101.747,19/09/1950
6,50ND075008_40A0111,14.9803,0.0,0,100,-1,-1,0,100,48.8991,-101.554,20/09/1950
7,50ND075009_40A0130,13.7386,0.0,0,100,-1,-1,0,100,48.9427,-101.587,20/09/1950
8,50ND075010_40A0131,15.3312,0.0,0,100,-1,-1,0,100,48.9153,-101.606,20/09/1950
9,S1964TN157003_40A0158,4.67086,0.0,0,100,-1,-1,0,100,-300.0,-300.0,01/03/1964


Results here:

(full) https://drive.google.com/file/d/1v8YxVTfFq0x4MhkRtBLtFGgBQIC_9RFI/view?usp=sharing

(clipped) https://drive.google.com/file/d/1Gw8uGphn3Mup8OPABXaumtgQEg37UIj6/view?usp=sharing
